#Markdown

```plaintext
RL/
├── env/
│   ├── grid.py
│   ├── gridln.py
│   ├── gridnn.py
│   ├── mountainln.py
│   ├── robot.py           ← Gazebo interface and Environment
│   └── robot_old.py
├── rl/
│   ├── dp.py              ← Dynamic programming
│   ├── rl.py              ← Core RL logic
│   ├── rlln.py            ← Linear approximation model
│   ├── rlnn.py            ← Non-linear model 
│   └── rlselect.py        ← "Runs" code for running experimental trials comparing
```

# Imports / Constants

In [1]:
%matplotlib inline
import torch
import numpy as np
from env.robot import *
import numpy as np
from math import pi
from time import sleep
#from tqdm import tqdm

from tqdm.notebook import tqdm
import sys
import termios
import tty
import select
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import time
import matplotlib.pyplot as plt
import numpy as np


#ACTIONS - make the code easier to read
FORWARDS = 1
LEFT = 0
RIGHT = 2

#Really want it deterministic
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#GPU support?
print(torch.cuda.is_available()) 
print(torch.cuda.get_device_name(0)) 

True
NVIDIA GeForce RTX 3080 Ti


### Common / Helper Functions

In [2]:
def print_robot_state(env: RobEnv):
    #note: env.x and env.y are rounded to 1dp
    print (f"Odom. Pos:[{env.x},{env.y}] Yaw:{env.θ}")

# Connect to ROS / Configure Environment

1) Launch simulation environment
2) init ros (connect to ROS DDS eventing)
3) create environment

In [3]:

# Start Gazebo
# Assessment world:
# ros2 launch turtlebot3_gazebo turtlebot3_assessment2.launch.py

# Other worlds
# ros2 launch turtlebot3_gazebo turtlebot3_simple.launch.py 
# ros2 launch turtlebot3_gazebo empty_world.launch.py
# ros2 launch turtlebot3_gazebo turtlebot3_world.launch.py
# ros2 launch turtlebot3_gazebo turtlebot3_house.launch.py
# rviz2 

if not ros.ok():
    ros.init()


#Values (see calibration below for how these values were picked for the most repeatable settings)
θspeed = pi/3.5
speed = 10.0
n = 6

env = RobEnv(speed=speed, θspeed=θspeed, n=n, verbose=True)
env.reset()

nscans = get_nscans_LiDAR()
print(f"Num laser scans in sensor:'{nscans}'")
# set_nscans_LiDAR(nscans=64)

# accelerate_sim(speed=10)
# Note running on my machine (AMD Ryzen 9 7950X, 64GB RAM, 3080TI) I got a stable Real Time Factor of 1.00 in the simulation.
# I had problems if I set the real time target to more than this, and from reading up this is a knowm limitation in the old Gazebo environment
# the new Gazebo Sim properly abstracts time so there is a simulation time independent from wall clock time. This version doesn't play nicely unless it's 1:1
# See Simulation Speed in ROS/Gazebo   


print_robot_state(env)

speed  =  10.0
θspeed =  0.9
Num laser scans in sensor:'360'
Odom. Pos:[0.0,0.0] Yaw:0.07


In [4]:
## Reset env
env.reset()

4

In [5]:
#Test connection

for _ in range(10): env.step()

print_robot_state(env)

env.reset()

Odom. Pos:[0.7,0.1] Yaw:0.15


4

# Model 1: Action-value with linear function approximation

RL method explanation + justification

State representation

Reward function

Hyperparameter tuning

Learning curves + discussion

In [6]:
#Assumptions
#

## State representation

Explain design


In [ ]:
class vRobEnv(RobEnv):
    def __init__(self, **kw):
        super().__init__(**kw)
        self.nF = len(self.scans)
        print('state size(laser beams)=', self.nF)


    def nearly_atwall(self):
        # check only 2*rng front scans for collision, given the robot does not move backward
        rng = int(len(self.scans)*self.nscansρ//2)  # nscansρ//2 left and nscansρ//2 right 
        return np.r_[self.scans[-rng:], self.scans[:rng]].min() <= (self.min_range + 0.1) 
        # return self.scans.min() <= self.min_range

    # overridding reward_, 
    # you may use goal_dist, Δgoal_dist, θgoal_dist, Δθgoal_dist or at_wall and at_goal
    def reward_(self, a):
        reward = sum([
            -0.1, #Don't like steps

            -5 * self.nearly_atwall(), #Dont like getting too close to walls 

            -2 * self.Δgoal_dist, #Going towards the goal is good, away is bad
            #-0.5 * self.Δθgoal_dist / pi

            0.2 * (a == FORWARDS), # let's promote moving forward

            10 * self.atgoal(self.goal_dist) #Like goals
            ])
        
        if reward is None:
            print(f"BUG reward is None")
            return 0

        if self.verbose and reward>-1: print('reward =', reward)#; print(f'action = {a}')
        return reward
    
    # overriding state representation, you may only use the laser self.scans
    def s_(self):
        max, min = self.max_range, self.min_range
        # returns a normalise and descritised componenets
        return  1*(((self.scans - min)/(max - min))>=.5)


In [10]:
class FeatureExtractor:
    NUM_FEATURES = 6

    def __init__(self, 
                 near_threshold=0.5, 
                 wall_threshold=1.2, 
                 max_range=3.5,
                 min_range=0.0):

        #This implementation depends on 360 laser lines
        assert get_nscans_LiDAR() == 360        
        self.near_threshold = near_threshold
        self.wall_threshold = wall_threshold
        self.max_range = max_range
        self.min_range = min_range



    def extract_features(self, scan_data):
        scans = np.clip(scan_data, self.min_range, self.max_range)
        scans[np.isnan(scans)] = self.max_range

        n = len(scans)

        """
                FRONT [0]
                   ↑
     FRONT/LEFT [315]     FRONT/RIGHT [45]
          ↖                      ↗
LEFT [270] ←                     → RIGHT [90]
          ↙                      ↘
     BACK/LEFT [225]     BACK/RIGHT [135]
                   ↓
               BACK [180]
        """


        width_each_side = 5

        left_scan_range = scans[270-width_each_side:270+width_each_side]
        left_scan_average = np.mean(left_scan_range)

        front_scan_range = scans[np.r_[360-width_each_side:360, 0:width_each_side]]
        front_scan_average = np.mean(front_scan_range)
        
        right_scan_range = scans[90-width_each_side:90+width_each_side]
        right_scan_average = np.mean(right_scan_range)

        def is_wall_detected(avg_distance):
            return avg_distance < self.wall_threshold
                
        def is_wall_too_near(avg_distance):
            return avg_distance < self.near_threshold
        
        labels = []
        features = []

        labels.append('Left Wall Ahead')
        features.append(is_wall_detected(left_scan_average))
        labels.append('Left Wall Getting Close')
        features.append(is_wall_too_near(left_scan_average))

        labels.append('Front Wall Ahead')
        features.append(is_wall_detected(front_scan_average))
        labels.append('Front Wall Getting Close')
        features.append(is_wall_too_near(front_scan_average))

        labels.append('Right Wall Ahead')
        features.append(is_wall_detected(right_scan_average))
        labels.append('Right Wall Getting Close')
        features.append(is_wall_too_near(right_scan_average))

        #labels.append('Pillar Recognised')
        #features.append(self.detect_cylinder_like(scans))
        
        assert len(labels) == self.NUM_FEATURES

        return labels, np.array(features, dtype=np.float32), left_scan_average, front_scan_average, right_scan_average

    def detect_cylinder_like(self, scan_data, range_threshold=0.5, cluster_width=5, variance_threshold=0.05):
        for i in range(len(scan_data) - cluster_width):
            window = scan_data[i:i+cluster_width]
            if np.all(window < range_threshold) and np.var(window) < variance_threshold:
                return 1
        return 0
    

class vRobEnvCornerDetector(RobEnv):
    def __init__(self, **kw):
        super().__init__(**kw)
        self.nF = FeatureExtractor.NUM_FEATURES # len(self.scans)
        print('state size=', self.nF)

    # overridding reward_, 
    # you may use goal_dist, Δgoal_dist, θgoal_dist, Δθgoal_dist or at_wall and at_goal
    #def reward_(self, a):
        # reward = sum([-self.Δgoal_dist*(a==1), .1*(a==1), .1*(a!=1),-self.Δθgoal_dist*(a!=1)/pi, -2*self.at_wall, 5*self.at_goal])
        # reward = sum([-1, -1*(a != 1)*(not self.at_goal), 1*self.at_goal])/3  #-1*self.at_wall,

        #-1 FOR EVERY STEP, 10 IF AT THE GOAL!!!
    #    reward = sum([-1, 10*self.at_goal])
    #    if self.verbose and reward>-1: print('reward =', reward)#; print(f'action = {a}')
    #    return reward

    #def reward_(self, a):
    #    # reward = sum([-self.Δgoal_dist*(a==1), .1*(a==1), .1*(a!=1),-self.Δθgoal_dist*(a!=1)/pi, -2*self.at_wall, 5*self.at_goal])
    #    # reward = sum([-1, -1*(a != 1)*(not self.at_goal), 1*self.at_goal])/3  #-1*self.at_wall,
    #    reward = sum([-1, 10*self.at_goal])
    #    if self.verbose and reward>-1: print('reward =', reward)#; print(f'action = {a}')
    #    return reward

    #reward = sum([-1, -1*(a != 1)*(not self.at_goal), 1*self.at_goal])/3  #-1*self.at_wall,
    #def reward_(self, a):
    #    reward = sum([
    #        -1,  #penalty per step
    #        -1*(a != 1)*(not self.at_goal), #penalty for turning when not at goal, 
    #        1*self.at_goal #reaching the goal
    #        ])/3 #???? Keep it moderate?
        
    #    if self.verbose and reward > -1:
    #        print(f'reward:{reward} = SUM (-1,{-10 * self.at_wall},{10 * self.at_goal},{0.2 * (a == FORWARDS)})' )
    #    return reward
    
    def nearly_atwall(self):
        # check only 2*rng front scans for collision, given the robot does not move backward
        rng = int(len(self.scans)*self.nscansρ//2)  # nscansρ//2 left and nscansρ//2 right 
        return np.r_[self.scans[-rng:], self.scans[:rng]].min() <= (self.min_range + 0.1) 
        # return self.scans.min() <= self.min_range

    # you may use goal_dist, Δgoal_dist, θgoal_dist, Δθgoal_dist or at_wall and at_goal
    def reward_(self, a):
        reward = sum([
            -0.1, #Don't like steps

            -5 * self.nearly_atwall(), #Dont like getting too close to walls 

            -2 * self.Δgoal_dist, #Going towards the goal is good, away is bad
            #-0.5 * self.Δθgoal_dist / pi

            0.2 * (a == FORWARDS), # let's promote moving forward

            10 * self.atgoal(self.goal_dist) #Like goals
            ])
        
        if reward is None:
            print(f"BUG reward is None")
            return 0

        if self.verbose and reward>-1: print('reward =', reward)#; print(f'action = {a}')
        return reward


    # overriding state representation, you may only use the laser self.scans
    def s_(self):

        #walls (front, left, right)

        #corners (front+left, front+right)

        #and pillar signal (from above)?

        self.fe = FeatureExtractor(
            near_threshold=0.5,
            wall_threshold=1.2,
            max_range=self.max_range,
            min_range=self.min_range)
        
        _, features_detected, _, _, _ = self.fe.extract_features(env.scans)
        
        # returns a normalise and descritised componenets
        return  features_detected
#

In [11]:
#STOP


hyperparameters = {
    'ε': 0.5,
    'εmin': 0.05,
    'dε': 0.0005,
    'α': 0.05,          
    'γ': 0.99,          
    'λ': 0.6,

    'verbose': False,
    
    # Robot Environment params
    'θspeed': pi / 6, 
    'speed': 0.3,        
    'n': 8             
}


venv = vRobEnvCornerDetector(
    speed=hyperparameters['speed'],
    θspeed=hyperparameters['θspeed'],
    n=hyperparameters['n'],
    verbose=hyperparameters['verbose']
)
venv.reset()

vqlearn = Sarsaλ(
    env=venv,
    α=hyperparameters['α'],          
    γ=hyperparameters['γ'],          
    λ=hyperparameters['λ'],           
    ε=hyperparameters['ε'],            
    εmin=hyperparameters['εmin'],        
    dε=hyperparameters['dε'],        
    q0=0,             
    Tstar=0,          
    max_t=1500,       
    episodes=200,     
    self_path='SarsaLambda.test008.pkl',
    seed=1,
    **demoGame()      # keep this
)

%time vqlearn.interact(resume=False, save_ep=True)


[WARN] [1745759448.322056839] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


speed  =  0.3
θspeed =  0.52
state size= 6
(6,), None
(6,), None
(6,), 0
(6,), 2
(6,), 2
(6,), 0
(6,), 2
(6,), 1
(6,), 1
(6,), 2
(6,), None
(6,), 1
(6,), 2
(6,), 2
(6,), 1
(6,), None
(6,), 2
(6,), 0
(6,), 0
(6,), 2
(6,), None
(6,), 0
(6,), 1
(6,), 1
(6,), 0
(6,), None
(6,), 1
(6,), 1
(6,), 1
(6,), 1
(6,), None
(6,), 1
(6,), 1
(6,), 1
(6,), 1
(6,), None
(6,), 1
(6,), 2
(6,), 2
(6,), 1
(6,), None
(6,), 2
(6,), 0
(6,), 0
(6,), 2
(6,), None
(6,), 0
(6,), 0
(6,), 0
(6,), 0
(6,), None
(6,), 0
(6,), 2
(6,), 2
(6,), 0
(6,), None
(6,), 2
(6,), 1
(6,), 1
(6,), 2
(6,), 1
(6,), 2
(6,), 2
(6,), 1
(6,), None
(6,), 2
(6,), 2
(6,), 2
(6,), 2
(6,), None
(6,), 2
(6,), 0
(6,), 0
(6,), 2
(6,), None
(6,), 0
(6,), 1
(6,), 1
(6,), 0
(6,), None
(6,), 1
(6,), 2
(6,), 2
(6,), 1
(6,), None
(6,), 2
(6,), 0
(6,), 0
(6,), 2
(6,), None
(6,), 0
(6,), 2
(6,), 2
(6,), 0
(6,), None
(6,), 2
(6,), 2
(6,), 2
(6,), 2
(6,), None
(6,), 2
(6,), 0
(6,), 0
(6,), 2
(6,), None
(6,), 0
(6,), 0
(6,), 0
(6,), 0
(6,), None
(6,), 0
(6,

KeyboardInterrupt: 

In [ ]:
stop here

#set_nscans_LiDAR(nscans=64)
#sdfdsfs

SyntaxError: invalid syntax (3964023741.py, line 1)

In [ ]:
'''
 def print_V(self, agent):
        s = self.s_()        
        q = agent.Q_(s)      
        π = agent.π(s)      
        V = np.dot(q, π)     
        print("Current V(s) =", V)


θspeed = pi # pi/3.5
speed = 15.0
n = 10

venv = vRobEnv(speed=speed, θspeed=θspeed, n=n,verbose=True)

#vqlearn = Qlearn(env=venv, α=1e-4, q0=0, ε=.0, \
#                 max_t=1000, episodes=100, \
#                 self_path='test003.pkl',\
#                 seed=1, **demoGame())

#commit_ep=0, ε=.1, εmin=0.01, dε=1, εT=0, q0=0, Tstar=0,


#λ eligibility trace delay
#
vqlearn = Sarsaλ(
    env=venv,
    α=0.01,
    γ=0.99,
    λ=0.8,
    ε=0.2,
    εmin=0.05,
    dε=1e-4,
    q0=0,
    Tstar=0,
    max_t=200, 
                 episodes=100, 
                 self_path='SarsaLambda.test004.pkl',
                 seed=1, 
                 **demoGame())

%time vqlearn.interact(resume=resume, save_ep=True) 
'''


ε=0.5,
εmin=0.05,
dε=0.0005


θspeed = pi / 6  
speed = 0.3        
n = 8             

venv = vRobEnv(speed=speed, θspeed=θspeed, n=n, verbose=False)
venv.reset()
'''
vqlearn = Sarsaλ(
    env=venv,
    α=0.01,
    γ=0.99,
    λ=0.8,
    ε=0.2,
    εmin=0.05,
    dε=0.002,            
    q0=0,
    Tstar=0,
    max_t=1500,           
    episodes=200,        
    self_path='SarsaLambda.test007.pkl',
    seed=1,
    **demoGame()
)
'''

vqlearn = Sarsaλ(
    env=venv,
    α=0.05,           # stronger learning rate
    γ=0.99,           # good (keep discount high)
    λ=0.6,            # slightly less aggressive credit spreading
    ε=0.5,            # start with much more exploration
    εmin=0.05,        # keep same
    dε=0.0005,        # decay slower, explore longer
    q0=0,             # good (no bias)
    Tstar=0,          # good (always on-policy)
    max_t=1500,       # good (enough steps)
    episodes=200,     # good (you can later extend if needed)
    self_path='SarsaLambda.test008.pkl',
    seed=1,
    **demoGame()      # keep this (your settings for the environment)
)

''' def print_V(self, agent):
        s = self.s_()        
        q = agent.Q_(s)      
        π = agent.π(s)      
        V = np.dot(q, π)     
        print("Current V(s) =", V)'''

%time vqlearn.interact(resume=False, save_ep=True)

'''
---------------------------------------------------------------------------
IndexError                                Traceback (most recent call last)
File <timed eval>:1

File ~/git/turtlebot-as2/rl/rl.py:192, in MRP.interact(self, train, resume, save_ep, episodes, grid_img, **kw)
    189 self.t += 1
    190 self.t_+= 1
--> 192 rn,sn, a,an, done = self.step(s,a, self.t)  # takes a step in env and store tarjectory if needed
    193 self.online(s, rn,sn, done, a,an) if train else None # to learn online, pass a one step trajectory
    195 self.Σr += rn

File ~/git/turtlebot-as2/rl/rl.py:153, in MRP.step_an(self, s, a, t)
    151 if self.skipstep: return 0, None, None, None, True
    152 sn, rn, done, _ = self.env.step(a)
--> 153 an = self.policy(sn)
    155 # we added s=s for compatibility with deep learning later
    156 self.store_(s=s, a=a, rn=rn, sn=sn, an=an, done=done, t=t)

File ~/git/turtlebot-as2/rl/rl.py:487, in MDP.<locals>.MDP.εgreedy(self, s)
    484 if self.dε < 1: self.ε = max(self.εmin, self.ε*self.dε)              # exponential decay
    485 if self.εT > 0: self.ε = max(self.εmin, self.ε0 - self.t_ / self.εT) # linear      decay
--> 487 return self.greedy(s) if rand() > self.ε else randint(0, self.env.nA)

File ~/git/turtlebot-as2/rl/rl.py:477, in MDP.<locals>.MDP.greedy(self, s)
    474 # print(s)
    475 # print(Qs)
    476 if Qs.shape[0]==1: raise ValueError('something might be wrong number of actions ==1')
--> 477 return choices(np.where(Qs==Qs.max())[0])[0]

File /usr/lib/python3.10/random.py:519, in Random.choices(self, population, weights, cum_weights, k)
    517     floor = _floor
    518     n += 0.0    # convert to float for a small speed improvement
--> 519     return [population[floor(random() * n)] for i in _repeat(None, k)]
    520 try:
    521     cum_weights = list(_accumulate(weights))

File /usr/lib/python3.10/random.py:519, in <listcomp>(.0)
    517     floor = _floor
    518     n += 0.0    # convert to float for a small speed improvement
--> 519     return [population[floor(random() * n)] for i in _repeat(None, k)]
    520 try:
    521     cum_weights = list(_accumulate(weights))

IndexError: index 0 is out of bounds for axis 0 with size 0

FIXED BY EDITING rl.py

#------------------------------------- add some more policies types 易-------------------------------
        # useful for inheritance, gives us a vector of actions values
        def Q_(self, s=None, a=None):

            #Originally return self.Q[s] if s is not None else self.Q

            if s is None:
                return self.Q  
            
            #just initialise to 0 for now, not sure how to handle this.
            if s not in self.Q:
                self.Q[s] = np.zeros(self.env.nA) 
            
            return self.Q[s]

'''



speed  =  0.3
θspeed =  0.52
state size(laser beams)= 60


[WARN] [1745753316.947744027] [rcl.logging_rosout]: Publisher already registered for provided node name. If this is due to multiple nodes with the same name then all logs for that logger name will go out over the existing publisher. As soon as any node with that name is destructed it will unregister the publisher, preventing any further logs for that name from being published on the rosout topic.


KeyboardInterrupt: 

"\n---------------------------------------------------------------------------\nIndexError                                Traceback (most recent call last)\nFile <timed eval>:1\n\nFile ~/git/turtlebot-as2/rl/rl.py:192, in MRP.interact(self, train, resume, save_ep, episodes, grid_img, **kw)\n    189 self.t += 1\n    190 self.t_+= 1\n--> 192 rn,sn, a,an, done = self.step(s,a, self.t)  # takes a step in env and store tarjectory if needed\n    193 self.online(s, rn,sn, done, a,an) if train else None # to learn online, pass a one step trajectory\n    195 self.Σr += rn\n\nFile ~/git/turtlebot-as2/rl/rl.py:153, in MRP.step_an(self, s, a, t)\n    151 if self.skipstep: return 0, None, None, None, True\n    152 sn, rn, done, _ = self.env.step(a)\n--> 153 an = self.policy(sn)\n    155 # we added s=s for compatibility with deep learning later\n    156 self.store_(s=s, a=a, rn=rn, sn=sn, an=an, done=done, t=t)\n\nFile ~/git/turtlebot-as2/rl/rl.py:487, in MDP.<locals>.MDP.εgreedy(self, s)\n    484 i

In [ ]:
resume = False

if resume:
    vqlearn = Qlearn.selfload(self_path='vQlearn_exp')
    vqlearn.env = venv
    vqlearn.episodes = 105 # extend sthe number of episodes

# saving the object after each episode for retrieval in case of a crash 
%time vqlearn.interact(resume=resume, save_ep=True) 

reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.3333333333333333 = SUM (-1,0,0,0.2)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.3333333333333333 = SUM (-1,0,0,0.2)
reward:-0.3333333333333333 = SUM (-1,0,0,0.2)
reward:-0.3333333333333333 = SUM (-1,0,0,0.2)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.3333333333333333 = SUM (-1,0,0,0.2)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.3333333333333333 = SUM (-1,0,0,0.2)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (-1,0,0,0.0)
reward:-0.6666666666666666 = SUM (

KeyboardInterrupt: 

Model 2: Either policy gradient or value-based with non-linear function approximation

I have also provided you with the ability to easily create a simple, fully connected neural network in the nnMRP class. You do not need to do a lot; the size of the input dictates the choice between a CNN-based model and a fully connected neural network. You may use either in your project, but using a traditional network, not a CNN, is easier and less time-consuming. The CNN-based one is for learning from the pixels of an input image; we are just using the laser reading in our project.



I’ve also made slight changes to rl.rl.py to enable storing and retrieving an object (via pickle), which can be useful in case of unexpected crashes—something not uncommon in robotics.

Please ensure you download turtlebot3.zip as well to guarantee the environment runs smoothly without any missing files. If you run into any issues, feel free to reach out.

# References

- Núñez, P., Vazquez-Martin, R., Bandera, A., and Romero-Gonzalez, C. (2015) ‘Feature extraction from laser scan data based on curvature estimation for mobile robotics’, *Robotics and Autonomous Systems*, 70, pp. 103–114. Available at: [https://robolab.unex.es/wp-content/papercite-data/pdf/feature-extraction-from-laser.pdf](https://robolab.unex.es/wp-content/publicaciones/2006/Nunez%20Trujillo,%20Vazquez-Martin,%20del%20Toro,%20Bandera%20%7C%20Feature%20extraction%20from%20laser%20scan%20data%20based%20on%20curvature%20estimation%20for%20mobile%20robotics.pdf) (Accessed: 26 April 2025).

- Ramos, J., Rocha, R., and Dias, J. (2022) ‘Efficient approach for extracting high-level B-spline features from laser scan data’, *Sensors*, 22(24), 9737. Available at: [https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9737135/](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9737135/) (Accessed: 26 April 2025).

- Shen, S., Michael, N., and Kumar, V. (2012) ‘Method for corner feature extraction from laser scan data’, *ResearchGate*. Available at: [https://www.researchgate.net/publication/288577925_Method_for_corner_feature_extraction_from_laser_scan_data](https://www.researchgate.net/publication/288577925_Method_for_corner_feature_extraction_from_laser_scan_data) (Accessed: 26 April 2025).

- Stack Overflow (2019) ‘How can I detect the corner from 2D point cloud or LiDAR scanned data?’, *Stack Overflow*. Available at: [https://stackoverflow.com/questions/59049990/how-can-i-detect-the-corner-from-2d-point-cloud-or-lidar-scanned-data](https://stackoverflow.com/questions/59049990/how-can-i-detect-the-corner-from-2d-point-cloud-or-lidar-scanned-data) (Accessed: 26 April 2025).


- CETI. (n.d.) *Simulation Speed in ROS/Gazebo*. Available at: [https://ceti.pages.st.inf.tu-dresden.de/robotics/howtos/SimulationSpeed.html](https://ceti.pages.st.inf.tu-dresden.de/robotics/howtos/SimulationSpeed.html) (Accessed: 26 April 2025).

- Furrer, F., Wermelinger, M., Naegeli, T., et al. (2021) ‘Dynamics and Control of Quadrotor UAVs: A Survey’, *IEEE Transactions on Robotics*, 37(5), pp. 1381–1400. Available at: [https://ieeexplore.ieee.org/document/9453594](https://ieeexplore.ieee.org/document/9453594) (Accessed: 26 April 2025).

- Perez-Perez, J., Jimenez, F. and Mata, M. (2023) ‘An Overview of Reinforcement Learning in Autonomous Driving: Fundamentals, Challenges, and Applications’, *Applied Sciences*, 13(12), p. 7202. Available at: [https://www.mdpi.com/2076-3417/13/12/7202](https://www.mdpi.com/2076-3417/13/12/7202) (Accessed: 26 April 2025).


# Appendicies

## Cool links / interesting reading: 
- https://github.com/hello-robot/stretch_ros/blob/master/stretch_funmap/README.md
- https://arxiv.org/pdf/2502.20607

## Miscelaneous Notes

### Setting up ROS
- https://emanual.robotis.com/docs/en/platform/turtlebot3/sbc_setup/
- https://ros2-industrial-workshop.readthedocs.io/en/latest/_source/navigation/ROS2-Turtlebot.html
- https://emanual.robotis.com/docs/en/platform/turtlebot3/navigation/
- https://emanual.robotis.com/docs/en/platform/turtlebot3/bringup/#bringup

### Multicast traffic (for DDS) through Windows FW to WSL2:
- https://eprosima-dds-router.readthedocs.io/en/latest/rst/examples/repeater_example.html#execute-example
- New-NetFirewallRule -Name 'WSL' -DisplayName 'WSL' -InterfaceAlias 'vEthernet (WSL (Hyper-V firewall))' -Direction Inbound -Action Allow
- New-NetIPAddress -InterfaceAlias 'vEthernet (WSL (Hyper-V firewall))' -IPAddress '192.168.1.217' -PrefixLength 24
- https://github.com/DanielBryars/multicast-test.git

### VM
- https://labs.azure.com/virtualmachines?feature_vnext=true